# Dynamics: simulation and control
This chapter focuses on the robot dynamics, for both simulating the behavior of the robot when forces are applied to it, and computing dynamic-aware control laws like computed torque.

Let's first do the basic imports.

In [1]:
import pinocchio
from pinocchio.utils import *
from numpy.linalg import norm, inv, pinv
from numpy import cos,sin
import time

## A robot hand for our test

We are going to use a 4-finger hand, whose model is defined in Python (no urdf model) using capsule volumes.
The hand is defined in the python code robot_hand.py. You can load and 
display it like this (don't forget to turn gepetto-viewer on with starting gepetto-gui).

In [2]:
from tp4.robot_hand import Robot
robot = Robot()
robot.display(robot.q0)

Take care that the hand is small: zoom in to see it in the window (or press the space bar).

Remember that the model rendered in the viewer is just a display, not a simulation (yet). You can pretty much achieve any movements you want in this display, because we are not enforcing (yet) any physical law. For example, if you want to move the fingers following any trajectories you like, you can do:

In [3]:
q = robot.q0.copy()

for i in range(100): # Put 1000 or 5000 if you want a longer move.
    for iq in range(3,robot.model.nq):
        q[iq] = -1+np.cos(i*1e-2*(1+iq/5))
    robot.display(q)
    time.sleep(2e-3)


## A QP solver
We will need a proper QP solver with inequality. QuadProg is a Python wrap of a nice Golub-based solver. Install it with PIP (pip install --user quadprog in the shell).

QuadProg main function is solve_qp. You have a bit of documentation using the Python help command *help(solve_qp)*. 


In [4]:
from quadprog import solve_qp

# help(solve_qp)

Here is a minimal example.

In [5]:
N = 6
M = 3

H = rand([N,N]); H = H*H.T
g = rand(N)

C = rand([M,N])
d = rand(M)

asarray = np.asarray
x,_,_,_,_,_ = solve_qp( asarray(H),asarray(g).T[0],
                        asarray(C).T,asarray(d).T[0] )


This example solves a QP problem of the form:
$$\min_x \frac{1}{2} x^T H x - g^T x$$
Under the constraints:
$$C x \ge d$$

## Forward unconstrained dynamics
### Computing the acceleration

In a first time, let's play with the dynamics without constraints (hence without quadprog).

Choosing an arbitrary joint torque $\tau_q$, pinocchio can compute the robot acceleration and integrate it.

The dynamic equation of the robot is $M a_q + b = \tau_q$, with $M$ the mass, $a_q$ the joint acceleration and $b$ the drift.
The mass matrix can be computed using *CRB* algorithm (function of q). The drift is computed using *RNE* algorithm (function of $q$, $v_q$ and $a_q$ with $a_q=0$). 


In [6]:
q   = rand(robot.model.nq)
vq  = rand(robot.model.nv)
aq0 = zero(robot.model.nv)
b = pinocchio.rnea(robot.model,robot.data,q,vq,aq0)  # compute dynamic drift -- Coriolis, centrifugal, gravity
M = pinocchio.crba(robot.model,robot.data,q)         # compute mass matrix M

print (np.linalg.norm(M-M.T))


0.0


These terms correspond to the inverse dynamics. They can be numerically inverted to compute the direct dynamics.

Using $M$ and $b$ computed by the above algorithms, and knowing a given set of joint torques $\tau_q$, how would you compute $a_q$ so that $M a_q + b = \tau_q$?


In [7]:
# %load tp4/solution_forward_dynamics.py
tauq = rand(robot.model.nv)
aq = inv(M)*(tauq-b)

The inverse-dynamics algorithm indeed compute the needed torques to achieve a given acceleration. We can use this function to double-check our result:

In [8]:
print(norm(pinocchio.rnea(robot.model,robot.data,q,vq,aq)-tauq))

1.066334393690002e-15


### Integrating the acceleration

Once aq as been computed, it is straight forward to integrate it to velocity using $v_q += a_q \Delta t$. 
Integration to joint position is more complex in general, as we saw for the mobile robot. Here, $n_q = n_v$ and everything is Euclinea, so a simple += integration would work, but since it is implemented in pinocchio, let's keep the good habits:

In [9]:
dt = 1e-2
vq += aq*dt
q = pinocchio.integrate(robot.model,q,vq*dt)

Now, you should be able to implement a first simulation (not a rendering any more) of the robot behavior when no torque is applied (tauq = 0).



In [10]:
# %load tp4/solution_tauq0.py

Now modify the simulator to encompass joint friction, i.e. the torque is opposite to the velocity with a friction coefficient $K_f$ (take $K_f=0.1$ as a start).

In [11]:
# %load tp4/solution_kf.py

## Proportional-derivative and computed torque

Now choose a reference joint position (possibly time varying, like in the hand example).
The joint torques can then be computed to track the desired position, with $\tau_q = -K_p (q-q^{des}) - K_v v_q$. Both gains $K_p$ and $K_v$ should be properly chosen. Optimal tracking is obtained with $K_v = 2 \sqrt{K_p}$. 
In general, a desired velocity is also tracked to avoid tracking errors.



In [12]:
# %load tp4/solution_pd.py

Here, there is a strong coupling between joints, due to the mass matrix that is not compensated in the simple PD law. In theory, the computed torques is to compute the joint torque by inverse dynamics from a reference joint acceleration. This boils down to canceling the simulation equation by choosing the proper terms in the control law. It is now very interesting to implement in case of perfect dynamics knowledge. It might be more interesting to study in case the simulation is done with the perfect M, while the control is computed with approximate M (for example, using only the diagonal terms of the mass matrix). Let's rather simulate contact.


## Collision checking

The robot hand is composed of capsules, i.e. level-set of constant distance to a segment. Collision checking and distances are then easy to implement. Pinocchio implement a complete and efficient collision checking based on the fast collision library -- FCL. For the scope of this work, we rely only on capsules whose collision are easy to compute, and we do the computation in Python. Look at the source code of robot_hand if you want to know more.

Collision checking are done for a set of collision pairs that must be specified to the robot.
The collision checking method indeed compute the distance between the two objects, along with the so-called witness points. A method can also be used to display them.


In [13]:
robot.collisionPairs.append([2,8])
robot.collisionPairs.append([2,11])
robot.collisionPairs.append([2,14])
robot.collisionPairs.append([2,16])

Compute distance between object 2 and 8, i.e the first (idx=0) collision pair: 

In [14]:
idx  = 0
pinocchio.forwardKinematics(robot.model,robot.data,robot.q0)
dist = robot.checkCollision(idx)

Display the collision pair by adding two disks at the witness points.

In [15]:
robot.displayCollision(idx,0)

As an exercice: randomly sample a configuration with collision. Then interpolate from the robot.q0 configuration toward the collision configuration, until finding the first frame where the collision happens. At that configuration, the two "witness" collision points should match.

In [16]:
# %load tp4/solution_collision.py


We will also need the jacobian of the distance, i.e. the matrix that is telling how the two witness points are moving in the configuration space. This jacobian is simply the jacobian of the first witness point minus the jacobian of the second witness point. It is given by the collisionJacobian method of the robot class:

In [17]:
J = robot.collisionJacobian(idx,q)

## Contact simulator

The jacobian is a 1xN matrix (row matrix) corresponding to the contact normal.
Take care that some information are stored in the visual objects when calling checkCollision, that are later used by collisionJacobian. You have to call collisionJacobian right after checkCollision, or the resulting jacobian might not be coherent.

For all collision pairs in contact (distance below 1e-3), the Jacobian must be collected and stacked in a single J matrix (which has as many rows as active constraints). Similarly, distances must be stacked in a vector (same number of rows as the jacobian).

Now, the joint acceleration is constrained by the contact constraint. It can be written as a minimization problem using Gauss principle
$$min \quad \frac{1}{2}  (\ddot{q} - \ddot{q}_0 )^T M (\ddot{q} - \ddot{q}_0 )$$
$$s.t. \quad J \ddot{q} > 0 $$
where qddot_0 is the free acceleration, i.e. the acceleration obtained in Question 2 where no constraint is active.

In theory, the acceleration should be above the "centrifugal" acceleration (i.e. the acceleration caused by joint velocity only, often written Jdot qdot) but we neglect it here.

In case of penetration or negative velocity, having only position acceleration is not enough. A "trick" is often to require the contact acceleration to be above a proportional depending of the penetration distance: J qddot >= -dist, with dist the vector of stacked distances. A better  solution to avoid penetration is to implement an impact model. The simplest one is the inelastic impact, where normal velocity is simply canceled at impact.
For that, remember inactive contact (i.e. those that were not in collision at previous simulation step).
When a collision pair is detected that was not previously active, project the current velocity on the null space of all contacts:
$$\dot q = \dot q - J^+ J \dot q$$



You are now all set up for implementing the contact simulator.
he complete loop should be as follows: tauq is computed from a PD tracking a time-varying joint position (question 3). After computing tauq, all collision pairs must be checked to find those with distances below 1e-3. Corresponding Jacobians must be computed and stacked. If a new collision as appeared, the joint velocity must be projected to nullify it. If not collision is active, the joint acceleration is computed from inverting the mass matrix (question 2). Otherwise, it is computed using QuadProg (question 4). The resulting acceleration is integrated twice (question 1) before displaying the robot starting a new simulation iteration.


## Homework
Implement a contact simulator, for a robot following a PD controller tracking a moving configuration. Submit the resulting script on the web application.